# NoSQL DBs in Generative AI

**MongoDB:** Used for storing structured/unstructured metadata, user prompts, model outputs, and other app-level data.

**Elasticsearch:** Used for semantic search or retrieval-augmented generation (RAG), where relevant context is retrieved and passed to the LLM to enhance response quality.

## AI Knowledge Assistant

🗣️ Accepts a user query

🔍 Uses an Elasticsearch retriever tool to fetch related docs

🧾 Optionally checks user history or context from MongoDB

🤖 Uses LangChain Agent to decide how to answer

💬 Generates a response using LLM

🧠 Stores chat in MongoDB


**Architecture**

User Query

   ↓

LangChain Agent

   ├── Tool 1: ElasticsearchRetrieverTool

   ├── Tool 2: MongoDBChatHistoryTool

   ↓

LLM generates answer using both tools

   ↓

Answer is saved to MongoDB


In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
import traceback

# from langchain.agents import initialize_agent
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import ElasticsearchStore
from pymongo import MongoClient
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch("http://localhost:9200")

In [3]:
INDEX_NAME = "product_docs_vector"

documents = [
    "To reset your password, go to the settings page and click on 'Forgot Password'.",
    "Our refund policy allows returns within 30 days of purchase, with the original receipt.",
    "All electronics come with a one-year manufacturer warranty unless specified otherwise.",
    "You can track your order using the tracking number sent to your email after shipping.",
    "Contact our support team 24/7 via live chat or by calling our toll-free number.",
    "Premium members receive free shipping and priority customer support.",
    "Your data is encrypted and stored securely as per industry standards.",
    "Multiple failed login attempts will temporarily lock your account for 15 minutes.",
    "You can change your communication preferences from your profile settings.",
    "Install our mobile app for a better experience and faster checkout."
]

In [15]:
embedding = OpenAIEmbeddings()

In [16]:
vectorstore = ElasticsearchStore.from_texts(
    texts=documents,
    embedding=embedding,
    index_name=INDEX_NAME,
    es_connection=es,
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [17]:
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["support_bot"]
chat_collection = db["chats"]

In [18]:
llm = ChatOpenAI(model="gpt-4")

AttributeError: module 'langchain' has no attribute 'verbose'

In [22]:
# ==== Tool 1: Hybrid Elasticsearch Retriever ====
class HybridESRetrieverTool(Tool):
    name: str = "Hybrid Elastic Search Retriever"
    description: str = "Retrieves product knowledge using hybrid vector + text search."

    def __init__(self, es_client: Elasticsearch, index_name: str):
        super().__init__(self.name, self._preform_action, self.description)
        self.es = es_client
        self.index = index_name
        self.embedding_model = OpenAIEmbeddings()

    def _preform_action(self, query: str) -> str | None:
        try:
            # Create embedding for the user query
            query_vector = self.embedding_model.embed_query(query)

            # Build hybrid search: text + vector
            search_body = {
                "size": 5,
                "query": {
                    "bool": {
                        "should": [
                            {
                                "match": {
                                    "content": {
                                        "query": query,
                                        "boost": 1.0
                                    }
                                }
                            },
                            {
                                "knn": {
                                    "embedding_vector": {
                                        "vector": query_vector,
                                        "k": 5,
                                        "num_candidates": 100
                                    }
                                }
                            }
                        ]
                    }
                }
            }

            res = self.es.search(index=self.index, body=search_body)
            hits = res.get("hits", {}).get("hits", [])

            if not hits:
                return "No relevant content found."

            results = []
            for hit in hits:
                source = hit.get("_source", {})
                content = source.get("content", "[No content found]")
                score = hit.get("_score", 0)
                results.append(f"Score: {score:.2f}\nContent:\n{content}")

            return "\n\n---\n\n".join(results)

        except Exception as e:
            return f"Error during hybrid search: {str(e)}\n{traceback.format_exc()}"

In [21]:
# ==== Tool 2: MongoDB Chat History ====
class MongoChatHistoryTool(Tool):
    name: str = "User Chat History Tool"
    description: str = "useful for checking last 3 interactions by a user"

    def __init__(self):
        """Initialize tool."""
        super().__init__(name=self.name, description=self.description, func=self._preform_action)

    @staticmethod
    def _preform_action(user_id: str):
        history = chat_collection.find({"user_id": user_id}).sort("timestamp", -1).limit(3)
        return "\n".join([f"User: {h['prompt']}\nBot: {h['response']}" for h in history])

In [ ]:
es_tool = HybridESRetrieverTool(es_client=es, index_name="product_docs")
mongo_tool = MongoChatHistoryTool()


tools = [
    es_tool,
    mongo_tool
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    return_intermediate_steps=True
)

In [23]:
def generate_response(user_id, prompt):
    system_prompt = f"The user ID is {user_id}. The user asked: '{prompt}'"
    final_response = agent.run(system_prompt)

    # Save to MongoDB
    chat_collection.insert_one({
        "user_id": user_id,
        "prompt": prompt,
        "response": final_response
    })

    return final_response

In [ ]:
response = generate_response(user_id="123", prompt="How do I reset my password?")
print(response)